# Day 11: Monkey in the Middle

[*Advent of Code 2022 day 11*](https://adventofcode.com/2022/day/11) and [*solution megathread*](https://redd.it/...)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/11/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F11%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [4]:
testdata = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1""".splitlines()

inputdata = downloaded['input'].splitlines()

In [5]:
from IPython.display import display


display(f'{inputdata[:10]} ... {len(inputdata)=}')

"['Monkey 0:', '  Starting items: 50, 70, 54, 83, 52, 78', '  Operation: new = old * 3', '  Test: divisible by 11', '    If true: throw to monkey 2', '    If false: throw to monkey 7', '', 'Monkey 1:', '  Starting items: 71, 52, 58, 60, 71', '  Operation: new = old * old'] ... len(inputdata)=55"

In [6]:
from __future__ import annotations
import operator


class Operation:
    def __init__(self: Operation,
                 line: str):
        right_side = line.split(' = ', 1)[1]
        tokens = right_side.split()
        assert len(tokens) == 3
        if tokens[1] == '*':
            self.operator = operator.mul
        else:  # tokens[1] == '+'
            self.operator = operator.add

        if tokens[2] == 'old':
            self.second_term = 0
        else:
            self.second_term = int(tokens[2])

    def __str__(self: Operation) -> str:
        if self.operator == operator.mul:
            if self.second_term == 0:
                return 'multiplied by itself'
            else:
                return f'multiplied by {self.second_term}'
        else:
            return f'increased by {self.second_term}'

    def __repr__(self: Operation) -> str:
        return f'"{self}"'

    def evaluate(self: Operation,
                 worry: int,
                 modulo: int = 0,
                 debug: bool = False) \
            -> int:
        if self.second_term == 0:
            output = self.operator(worry, worry)
        else:
            output = self.operator(worry, self.second_term)

        if debug:
            display(f'    Worry level is {self} to {output}.')

        # The principle here is that since Criteria.evaluate
        # only looks at whether the item is divisible by
        # divisor / modulo_base, and both and addition and
        # multiplication (but not the integer division of part
        # one) are invariant over that criteria, we may exploit
        # it to keep the magnitude down. Except it isn't working
        # for some reason...
        if modulo != 0:
            output %= modulo
        return output

In [7]:
class Criteria:
    def __init__(self: Criteria,
                 divisor: int,
                 if_true: int,
                 if_false: int):
        self.divisor = divisor
        self.if_true = if_true
        self.if_false = if_false

    def __str__(self: Criteria) -> str:
        return f'divisible by {self.divisor}'

    def __repr__(self: Criteria) -> str:
        return f'  Test: {self}\n' + \
            f'    If true: throw to monkey {self.if_true}\n' + \
            f'    If false: throw to monkey {self.if_false}'

    def evaluate(self: Criteria,
                 worry: int,
                 debug: bool = False) \
            -> int:
        result = worry % self.divisor == 0
        if debug:
            display('    Current worry level is' +
                    f'{" not" if not result else ""} ' +
                    f'divisible by {self.divisor}.')
        if result:
            return self.if_true
        else:
            return self.if_false

In [8]:
from typing import List, Iterable


class Monkey:
    def __init__(self,
                 index: int,
                 items: List[int],
                 operation: Operation,
                 criteria: Criteria):
        self.index = index
        self.items = items
        self.operation = operation
        self.criteria = criteria
        self.inspections = 0

    def __str__(self) -> str:
        return f'Monkey {self.index}: ' + \
            f'{", ".join(str(item) for item in self.items)}'

    def __repr__(self) -> str:
        return str(self) + f' ({self.operation}, {self.criteria})'

    def count_inspections(self):
        self.inspections += len(self.items)

    @classmethod
    def from_lines(cls, lines: Iterable[str]) -> Monkey:
        def take_after_colon(line: str) -> str:
            return line.split(': ', 1)[1]

        def take_last_int(line: str) -> int:
            return int(line.split()[-1])

        for line in lines:
            if line.find('Monkey') != -1:
                index = int(line.split(' ')[-1][:-1])
            elif line.find('Starting items:') != -1:
                items = [int(item)
                         for item
                         in take_after_colon(line).split(', ')]
            elif line.find('Operation:') != -1:
                operation = Operation(take_after_colon(line))
            elif line.find('Test:') != -1:
                divisor = take_last_int(line)
            elif line.find('If true:') != -1:
                if_true = take_last_int(line)
            elif line.find('If false:') != -1:
                if_false = take_last_int(line)
        criteria = Criteria(divisor,
                            if_true,
                            if_false)
        return cls(index, items, operation, criteria)

    @classmethod
    def from_dataset(cls, lines: Iterable[str]) -> List[Monkey]:
        monkeys: List[Monkey] = list()
        current_lines: List[str] = list()
        for line in lines:
            if line != '':
                current_lines.append(line)
            else:
                monkeys.append(cls.from_lines(current_lines))
                current_lines = list()
        monkeys.append(cls.from_lines(current_lines))
        return monkeys

In [9]:
def inspect_item(m: Monkey,
                 worry: int,
                 modulo: int = 0,
                 debug: bool = False) \
                     -> Tuple[int, int]:
    if debug:
        display(f'  Monkey inspects an item with a worry level of {worry}.')
    worry = m.operation.evaluate(
        worry=worry,
        modulo=modulo,
        debug=debug)
    if modulo == 0:
        worry //= 3
        if debug:
            display('    Monkey gets bored with item. Worry level is ' +
                    f'divided by 3 to {worry}.')
    recipient = m.criteria.evaluate(worry, debug)
    return recipient, worry

<cell>5: error: Name "Tuple" is not defined  [name-defined]
<cell>5: note: Did you forget to import it from "typing"? (Suggestion: "from typing import Tuple")


In [10]:
def monkey_turn(m: Monkey,
                monkeys: List[Monkey],
                modulo: int = 0,
                debug: bool = False):
    if debug:
        display(f'Monkey {m.index}:')
    m.count_inspections()
    for worry in m.items:
        recipient, new_worry = inspect_item(
            m=m,
            worry=worry,
            modulo=modulo,
            debug=debug)
        if debug:
            display(f'    Item with worry level {new_worry} is thrown ' +
                    f'to monkey {recipient}.')
        monkeys[recipient].items.append(new_worry)
    m.items.clear()

In [11]:
def monkeys_round(monkeys: List[Monkey],
                  reassured: bool = True,
                  debug: bool = False):
    if reassured:
        modulo = 0
    else:
        modulo = reduce(operator.mul,
                        [monkey.criteria.divisor
                         for monkey in monkeys])
    for monkey in monkeys:
        monkey_turn(m=monkey,
                    monkeys=monkeys,
                    modulo=modulo,
                    debug=debug)

<cell>7: error: Name "reduce" is not defined  [name-defined]


In [12]:
from functools import reduce


monkeys = Monkey.from_dataset(testdata)
for round in range(1, 21):
    if round == 1:
        debug = True
    else:
        debug = False
    monkeys_round(monkeys=monkeys,
                  debug=debug)
    if round in [*range(2, 11), 15, 20]:
        display(f'After round {round}, the monkeys are holding items with ' +
                'these worry levels:')
        for monkey in monkeys:
            display(f'{monkey}')
assert reduce(operator.mul,
              sorted(monkey.inspections
                     for monkey in monkeys)[-2:]) == 10605

'Monkey 0:'

'  Monkey inspects an item with a worry level of 79.'

'    Worry level is multiplied by 19 to 1501.'

'    Monkey gets bored with item. Worry level is divided by 3 to 500.'

'    Current worry level is not divisible by 23.'

...

'After round 20, the monkeys are holding items with these worry levels:'

'Monkey 0: 10, 12, 14, 26, 34'

'Monkey 1: 245, 93, 53, 199, 115'

'Monkey 2: '

'Monkey 3: '

In [13]:
monkeys = Monkey.from_dataset(inputdata)
for round in range(1, 21):
    monkeys_round(monkeys)
reduce(operator.mul,
       sorted(monkey.inspections for monkey in monkeys)[-2:])

102399

In [14]:
HTML(downloaded['part1_footer'])

## Part Two

In [15]:
HTML(downloaded['part2'])

In [16]:
monkeys = Monkey.from_dataset(testdata)
for round in range(1, 10001):
    monkeys_round(monkeys=monkeys,
                  reassured=False)
    if round in [1, 20, 1000, 2000, 10000]:
        display(f'== After round {round} ==')
        for monkey in monkeys:
            display(f'Monkey {monkey.index} inspected items ' +
                    f'{monkey.inspections} times.')
assert reduce(operator.mul,
              sorted(monkey.inspections
                     for monkey in monkeys)[-2:]) == 2713310158

'== After round 1 =='

'Monkey 0 inspected items 2 times.'

'Monkey 1 inspected items 4 times.'

'Monkey 2 inspected items 3 times.'

'Monkey 3 inspected items 6 times.'

...

'== After round 10000 =='

'Monkey 0 inspected items 52166 times.'

'Monkey 1 inspected items 47830 times.'

'Monkey 2 inspected items 1938 times.'

'Monkey 3 inspected items 52013 times.'

In [17]:
monkeys = Monkey.from_dataset(inputdata)
for round in range(1, 10001):
    monkeys_round(monkeys=monkeys,
                  reassured=False)
    if round in [1, 20, 1000, 2000, 10000]:
        display(f'== After round {round} ==')
        for monkey in monkeys:
            display(f'Monkey {monkey.index} inspected items ' +
                    f'{monkey.inspections} times.')
reduce(operator.mul,
       sorted(monkey.inspections
              for monkey in monkeys)[-2:])

'== After round 1 =='

'Monkey 0 inspected items 6 times.'

'Monkey 1 inspected items 5 times.'

'Monkey 2 inspected items 12 times.'

'Monkey 3 inspected items 2 times.'

...

'Monkey 4 inspected items 145361 times.'

'Monkey 5 inspected items 96277 times.'

'Monkey 6 inspected items 17356 times.'

'Monkey 7 inspected items 92434 times.'

23641658401

In [18]:
HTML(downloaded['part2_footer'])